# Incremental collection of BTC/USD trade(spot) data using COIN API

#### SPOT BTC/USD MARKET: BITSTAMP

In [2]:
# This script includes the functions that downloads and prepares the data sets
# that are intended to use


## COIN CAP API
import requests
import pandas as pd
import numpy as np
import datetime
import os
api_key = "B90AC06B-4BCB-4FC2-B3BA-CBBABCF63791"


# def get_hist(market, )


url = 'https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history?'
headers = {'X-CoinAPI-Key' : api_key}

## Helper functions

In [3]:
def save_csv(df):
    '''
    Assumes the data has the generated time columns: time_day
    '''
    df.to_csv('BITSTAMP_TRADE_SPOTBTC_'+ str(df['time_day'].min())+ '__' + str(df['time_day'].max()) + '.csv')

In [4]:

def make_integer_time_index(time_exch, config=None):
    '''
    Creates integer time index from 'time_exchange' column returned in format of Coin API
    
    This function is designed to be called with pd.Series.apply(lambda x: func(x, config=config)), x is the value of 'time_exch' passed on by apply function,
    while what kind of index will be created should be specified by user via 'config' param.
    
        args:
            
            * time_exch: this is passed by .apply function from pandas
            * config (str): user specifies which time index to create  as '[agg_type]_[agg_freq]':
                - agg_type: 
                    = cycle:  only return the cycle. ( example if freq is day, then return day of the month example: 1, 2, 24, 30)
                    = bin: concatenate starting from largest freq(year) until specified freq (allows easy binning with this freq, hence the name 'bin')
    '''
    
    agg_type, agg_freq  = config.split('_')
    
    t = str(time_exch).split('T')

    year, month, day = t[0].split('-')
    hour, minute, second = t[1].split('.')[0].split(':')
    ymd_hms = { 'year': year, 'month':month, 'day': day, 'hour': hour, 'minute': minute, 'second': second  }

     
    if agg_type == 'cycle':
        return(int(ymd_hms[agg_freq]))
    
    elif agg_type == 'bin':
        bin_time = ''
        for freq, freq_value in ymd_hms.items():
            bin_time += freq_value
            if freq == agg_freq:
                break
        return(int(bin_time))

    
def make_all_time_indexes(df):             
    time_agg_inds = []
    
    for agg_type in ['cycle', 'bin']:
        for agg_freq in ['year','month', 'day', 'hour', 'minute','second']:
            print(agg_type, agg_freq)
            if agg_type == 'cycle':
                config = str(agg_type+'_'+agg_freq)
                df[agg_freq] = df['time_exchange'].apply(lambda x: make_integer_time_index(x, config=config))
                time_agg_inds.append(agg_freq)
            else:
                config = str('bin_'+agg_freq)
                df['time_'+agg_freq] = df['time_exchange'].apply(lambda x: make_integer_time_index(x, config=config))
                time_agg_inds.append('time_'+agg_freq)
    df['date'] = pd.to_datetime(df['time_exchange']).dt.date
    return df
    

## Load existing(cumulative) data / Add new / Save all together csv

In [5]:
df_saved = pd.read_csv('../../Data/Coins/BTC_USD/Trade_Data/BITSTAMP_TRADE_SPOTBTC_cum.csv' , index_col='uuid')
df_saved.shape

(799117, 19)

In [6]:
sum([c[-2]=='0' for c in df_saved['time_exchange'].values])

739189

##### TODO ADD DATE MANIPULATION FOR STARTING FROM NEXT DAY

In [7]:
# Manual day addition until above code fixed

first_day_of_new = df_saved['time_exchange'].max()
first_day_of_new = first_day_of_new[:-2]+'1Z'
last_day_of_new = '2016-07-01'

In [8]:
print(first_day_of_new)
print(last_day_of_new)

2016-06-12T21:38:35.0000001Z
2016-07-01


#### Get JSON data for next day(s)

In [9]:
url = 'https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history?time_start='+ first_day_of_new + '&time_end=' + last_day_of_new + '&limit=100000'

# # url = 'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?period_id=1MIN&time_start=2016-01-01T00:00:00'

response = requests.get(url, headers=headers)

if response.content[2:7] != b'error':
    df_new = pd.read_json(response.content).set_index('uuid')
    print(df_new.shape)
else:
    print(response.content)

(50000, 6)


### Make integer indexes

In [19]:
df_saved.columns

df_new = make_all_time_indexes(df_new)

cycle year
cycle month
cycle day
cycle hour
cycle minute
cycle second
bin year
bin month
bin day
bin hour
bin minute
bin second


In [23]:
print(df_saved['time_exchange'].unique().max())
print(df_new['time_exchange'].unique().min())
print(df_new['time_exchange'].unique().max())

2016-05-31T21:23:45.0000000Z
2016-05-31T21:24:17.0000000Z
2016-06-12T21:38:35.0000000Z


In [24]:
# from datetime import datetime as dt

# dt.strptime('2016-04-10T22:02:50.73000Z', '%Y-%m-%dT%H:%M:%S.%fZ')

### Join with old and save

In [25]:
df_both = pd.concat([df_saved, df_new])
print(df_both.shape)
df_both.head()

(799117, 19)


,price,size,symbol_id,taker_side,time_coinapi,time_exchange,year,month,day,hour,minute,second,time_year,time_month,time_day,time_hour,time_minute,time_second,date
uuid,,,,,,,,,,,,,,,,,,,
ec6fba23-29c7-4742-85b1-117b1e9e4574,430.89,0.080631,BITSTAMP_SPOT_BTC_USD,UNKNOWN,2016-01-01T00:01:01.2570000Z,2016-01-01T00:01:01.1500000Z,2016,1,1,0,1,1,2016,201601,20160101,2016010100,201601010001,20160101000101,2016-01-01
e435fd12-b49b-4334-9fff-9f37d8b2ce82,430.89,0.096757,BITSTAMP_SPOT_BTC_USD,UNKNOWN,2016-01-01T00:02:14.0030000Z,2016-01-01T00:02:13.8870000Z,2016,1,1,0,2,13,2016,201601,20160101,2016010100,201601010002,20160101000213,2016-01-01
df51e2de-1d76-4182-a269-7ef59c0ca1b3,430.89,0.046500,BITSTAMP_SPOT_BTC_USD,UNKNOWN,2016-01-01T00:02:46.5800000Z,2016-01-01T00:02:46.4830000Z,2016,1,1,0,2,46,2016,201601,20160101,2016010100,201601010002,20160101000246,2016-01-01
4c313843-3ed5-4222-9f01-597566506f26,430.19,0.200000,BITSTAMP_SPOT_BTC_USD,UNKNOWN,2016-01-01T00:04:32.6400000Z,2016-01-01T00:04:32.5200000Z,2016,1,1,0,4,32,2016,201601,20160101,2016010100,201601010004,20160101000432,2016-01-01
7e65517a-29ec-4cea-a625-0c080bce3921,430.89,0.432161,BITSTAMP_SPOT_BTC_USD,UNKNOWN,2016-01-01T00:04:36.4800000Z,2016-01-01T00:04:36.3600000Z,2016,1,1,0,4,36,2016,201601,20160101,2016010100,201601010004,20160101000436,2016-01-01


In [27]:
df_both.shape[0] == df_both.index.nunique()

True

In [28]:
all(df_both.isna().mean() == 0)

True

In [32]:
df_both.to_csv('../../Thesis_Workspace/Data/Coins/BTC_USD/Trade_Data/BITSTAMP_TRADE_SPOTBTC_cum.csv')
# import os
# print(os.getcwd())

## backup save

In [15]:
last_month = df_both['time_month'].max()
last_month

201603

In [16]:
file_name = 'BITSTAMP_TRADE_SPOTBTC_cum_backup'+str(last_month)+'.csv'
file_name

'BITSTAMP_TRADE_SPOTBTC_cum_backup201603.csv'

In [17]:
df_both[df_both['time_month']==last_month].to_csv(file_name)